Consider the following equation
$$- u''  = f, \quad x\in \Omega$$
$$u = 0, \quad x \in \partial \Omega.$$
We choose the true solution $u = x(e^{-(x-\frac{1}{3})^2/K} - e^{-\frac{4}{9}/K})$ with $K = 0.01$ on the interval $\Omega = [-1,1]$.

By FOSLS method, we have 
$$\sigma = - u'.$$
And the loss function
$$G(\sigma, u; f) = \|\text{div }\sigma + u -f \|_{0,\Omega}^2 + \|\frac{1}{\varepsilon}\sigma + \varepsilon \nabla u\|_{0,\Omega}^2 + \beta \|u\|^2_{0,\Omega}$$

In [1]:
import time

def TicTocGenerator():
    # Generator that returns time differences
    ti = 0           # initial time
    tf = time.time() # final time
    while True:
        ti = tf
        tf = time.time()
        yield tf-ti # returns the time difference

TicToc = TicTocGenerator() # create an instance of the TicTocGen generator

# This will be the main function through which we define both tic() and toc()
def toc(tempBool=True):
    # Prints the time difference yielded by generator instance TicToc
    tempTimeInterval = next(TicToc)
    if tempBool:
        print( "Elapsed time: %f seconds.\n" %tempTimeInterval )

def tic():
    # Records a time in TicToc, marks the beginning of a time interval
    toc(False)

In [2]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torch.nn import functional as F
from torch import nn, optim
from math import exp

In [3]:
# initialize parameters
global k, dx, beta 
k, dx, beta = .01, .005, 200

In [4]:
def f(x):
    return -(2*exp(-(x - 1/3)**2/k)*(6*k + 2*x - 27*k*x - 12*x**2 + 18*x**3))/(9*k**2)
  
def g(x):
    return torch.tensor([0.], requires_grad=True)

def u_exact(x):
    return x*(exp(-1/k*(x-1/3)**2) - exp(-4/(9*k)))

def sigma_exact(x):
    return exp(-4/(9*k)) - exp(-(x - 1/3)**2/k) + (x*exp(-(x - 1/3)**2/k)*(2*x - 2/3))/k
  
sq = lambda x: x ** 2
vsq = np.vectorize(sq)

In [5]:
# compute H1 norm of true u and sigma
L = 0.
R = 1.
test_set =  np.arange(L, R+dx, dx)
u = np.vectorize(u_exact)(test_set)
ud = -np.vectorize(sigma_exact)(test_set)
u_h1 = np.sum(dx*(vsq(u) + vsq(ud)))
u_l2 = np.sum(dx*vsq(u))

sigma = np.vectorize(sigma_exact)(test_set)
sigmad = np.vectorize(f)(test_set)
sigma_h1 = np.sum(dx*(vsq(sigma) + vsq(sigmad)))
sigma_l2 = np.sum(dx*vsq(sigma))

print('u: H1 norm square: %.6f, L2 norm square: %.6f ' %(u_h1, u_l2))
print('sigma: H1 norm square: %.6f, L2 norm square: %.6f ' %(sigma_h1, sigma_l2))

u: H1 norm square: 1.500809, L2 norm square: 0.014239 
sigma: H1 norm square: 466.257229, L2 norm square: 1.486570 


In [6]:
class MuSigmaPde(nn.Module):
    def __init__(self, dimension, mesh = 24, neuron = 14):
        super(MuSigmaPde, self).__init__()

        self.xdim = dimension
        # Layer 1
        self.fc1mu = nn.Linear(dimension, mesh)
        self.fc1sig = nn.Linear(dimension, mesh)
        # Layer 2
        self.fc2mu = nn.Linear(mesh, neuron)
        self.fc2sig = nn.Linear(mesh, neuron)
        # Layer 3
        self.fc3mu = nn.Linear(neuron, neuron)
        self.fc3sig = nn.Linear(neuron, neuron)
        # Layer 4
        self.fc4mu = nn.Linear(neuron, 1)
        self.fc4sig = nn.Linear(neuron, dimension)

    def forward(self, x):  #Sigmoid activation function
        assert(len(x.shape) == 1 and x.shape[0] == self.xdim)
        y_mu =  torch.sigmoid(self.fc2mu(torch.sigmoid(self.fc1mu(x))))
        y_sig =  torch.sigmoid(self.fc2sig(torch.sigmoid(self.fc1sig(x))))
        mu =  self.fc4mu(torch.sigmoid(self.fc3mu(y_mu)))
        sigma = self.fc4sig(torch.sigmoid(self.fc3mu(y_sig)) )
        return mu, sigma
      
    def net_grad(self, x):
        mu_center, sigma_center = self.forward(x)
        mu_forward, sigma_forward = self.forward(x - .5*dx)

        mu_grad_forward = (mu_center - mu_forward)/(.5*dx)
        sigma_grad_forward = (sigma_center - sigma_forward)/(.5*dx)
    
        return mu_grad_forward, sigma_grad_forward
    
    def loss_function_bulk(self, x):   #energy functional
        mu, sigma = self.forward(x)
        mu_grad, sigma_grad = self.net_grad(x)
        LSE = 0.5*(mu_grad)**2-f(x)*mu
        return LSE

    def loss_function_surf(self, x):
        mu, sigma = self.forward(x)
        # Boundary condition penalty
        BCP = beta * (mu - g(x))**2
        return BCP

In [7]:
model = MuSigmaPde(dimension =1, mesh = 24, neuron = 14)

In [8]:
sum([p.numel() for p in model.parameters()])

1246

In [9]:
# change h for different number of quadrature points. There are 1/h=200 quadrature points in this case.

h = .005
epochs = 10000
L, R = 0., 1.
bulk_set, surf_set =  np.arange(L, R, h), [L, R]
loss_bulk_record, loss_surf_record = [], []
print('bulk points number %d \nsurface points number %d\ntest points number %d\ndx for difference in testing %.3g\ntrainging iteration %d' %(np.size(bulk_set), np.size(surf_set), np.size(test_set), dx, epochs))

bulk points number 200 
surface points number 2
test points number 201
dx for difference 0.005
trainging iteration 2000


In [10]:
optimizer = optim.Adam(model.parameters(), lr = 0.0005)

In [ ]:
tic()
local_min = 100.0
for j in range(epochs):
    loss_bulk = torch.zeros(1)
    loss_surf = torch.zeros(1)

    for point in bulk_set:
        x = torch.tensor([point+ 0.5*h])
        loss_bulk += h*model.loss_function_bulk(x)
               
    for point in surf_set:
        x = torch.tensor([point])
        loss_surf += model.loss_function_surf(x)

    # record each loss
    loss_bulk_record.append(loss_bulk.data[0])
    loss_surf_record.append(loss_surf.data[0])
    

    
    loss = loss_bulk + loss_surf
    if loss.item() < local_min:
        local_min = loss.item()
        torch.save(model.state_dict(),'./poisson_equationenergy_24_14_2')
    print('Train Epoch: {}, Loss: {:.6f}, loss bulk: {:.6f}, loss surf: {:.6f}'.format(j, loss.item(), loss_bulk.item(), loss_surf.item()))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
toc()    

In [16]:
model.load_state_dict(torch.load('./poisson_equationenergy_24_14_2'))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [17]:
mu_err_h1 = torch.zeros(1)
sigma_err_h1 = torch.zeros(1)
bdd_err = torch.zeros(1)
mu_err_l2 = torch.zeros(1)
sigma_err_l2 = torch.zeros(1)
G_relative = torch.zeros(1)
mu_err_semi = torch.zeros(1)

test_set =  np.arange(L, R+dx, dx)

for point in test_set:

    x = torch.tensor([point+ 0.5*dx])
    mu, sigma = model(x)
    mu_grad, sigma_grad = model.net_grad(x)
    loss=model.loss_function_bulk(x)

    # esitmate H1 norm error
    mu_diff = (mu - u_exact(x))**2  + (mu_grad + sigma_exact(x))**2
    mu_diff_simi = (mu_grad + sigma_exact(x))**2
    sigma_diff = (sigma - sigma_exact(x))**2 + (sigma_grad - f(x))**2
    mu_err_h1 += dx*mu_diff
    sigma_err_h1 += dx*sigma_diff
    
    # estimate L2 norm error
    mu_err_l2 += dx*(mu - u_exact(x))**2
    sigma_err_l2 += dx*(sigma - sigma_exact(x))**2
    
    # estimate H1 semi norm  error
    #mu_err_semi    = mu_err_h1 - mu_err_l2
    mu_err_semi += dx*mu_diff_simi
    sigma_err_semi = sigma_err_h1 - sigma_err_l2

H1_err = mu_err_h1 + sigma_err_h1  
H1_err_relative = ((H1_err)**(1/2)) / ((u_h1 + sigma_h1)**(1/2))
L2_err = mu_err_l2 + sigma_err_l2
L2_err_relative = ((L2_err)**(1/2)) / ((u_l2 + sigma_l2)**(1/2))
mu_err_h1_relative = (mu_err_h1/u_h1)**(1/2)
mu_err_l2_relative = (mu_err_l2/u_l2)**(1/2)
mu_err_semi_relative = (mu_err_semi/(sigma_l2))**(1/2)
sigma_err_h1_relative = (sigma_err_h1/sigma_h1)**(1/2)
sigma_err_l2_relative = (sigma_err_l2/sigma_l2)**(1/2)
sigma_err_semi_relative = (sigma_err_semi/(sigma_h1 - sigma_l2))**(1/2)
bdd_err += abs(mu - g(x))
G_rel = (loss)**(1/2)/((u_h1 + sigma_h1)**(1/2))

                              
print('u: L2_rel: {:.6f}, H1_semi_rel: {:.6f}'.format( mu_err_l2_relative.item(), mu_err_semi_relative.item()))
print('sigma: L2_rel: {:.6f}\n'.format(sigma_err_l2_relative.item()))
print('G_rel: {:.6f}\n'.format(G_rel.item()))

u: L2_rel: 0.094188, H1_semi_rel: 0.073567
sigma: L2_rel: 1.000882

G_rel: 0.000309



In [14]:
points = test_set
yt = np.zeros_like(points)
y_diff = np.zeros_like(points)
ymu = np.zeros_like(points)
ysig = np.zeros_like(points)
for i in range(len(points)):
    yt[i] = u_exact(points[i])
    y_diff[i] =  -exp(-(points[i]-1/3)**2/0.01)*(-200*points[i]**2 + (200/3)*points[i] + 1)
    ymu[i], ysig[i] = model(torch.tensor([points[i]]))

In [15]:
plt.plot(points, yt, label = 'u_true')
plt.plot(points, ymu, label = 'u_approximation')
plt.legend()

In [ ]:
plt.plot(points, y_diff, label = 'sigma_true')
plt.plot(points, ysig, label = 'sigma_approximation')
plt.legend()

In [ ]:
num = np.arange(1, len(loss_bulk_record)+1, 1)
plt.plot(num, loss_bulk_record)
plt.plot(num, loss_surf_record)
plt.plot(num, np.add(loss_bulk_record , loss_surf_record))